In [6]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
from colorthief import ColorThief

### Color Names Mapping

In [9]:
bdf= pd.read_csv("Datasets/BigColorData.csv")
sdf= pd.read_csv("Datasets/fashion-colors.csv")

In [10]:
def big_find_closest_color(r, g, b):
    distances = np.sqrt((bdf["redDecimal"] - r) ** 2 +
                        (bdf["greenDecimal"] - g) ** 2 +
                        (bdf["blueDecimal"] - b) ** 2)
    
    min_index = distances.idxmin()
    
    return bdf.loc[min_index, "name"]

In [11]:
def small_find_closest_color(r, g, b):
    distances = np.sqrt((sdf["R"] - r) ** 2 +
                        (sdf["G"] - g) ** 2 +
                        (sdf["B"] - b) ** 2)
    
    min_index = distances.idxmin()
    
    return sdf.loc[min_index, "Color Name"]

### Color Determination

In [12]:
def ignore_alpha(image_path):
    """Loads a PNG image and extracts only the non-transparent pixels."""
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    if image.shape[2] == 4:  # Check if it has an alpha channel
        rgb_image = image[:, :, :3]  # Extract RGB channels
        alpha_channel = image[:, :, 3]  # Extract alpha (transparency)
        clothing_pixels = rgb_image[alpha_channel > 0]  # Keep only opaque clothing pixels
        return clothing_pixels
    else:
        raise ValueError("Image does not have an alpha channel. Ensure it is a transparent PNG.")

In [14]:
def dominant_colors_kmeans(image_pixels, k=3):
    """Finds the number of dominant colors using K-Means clustering."""
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(image_pixels)
    counts = np.bincount(kmeans.labels_)
    unique_colors = len(set(map(tuple, np.round(kmeans.cluster_centers_))))  # Count unique cluster centers
    dominant=kmeans.cluster_centers_[np.argmax(counts)]
    return unique_colors, tuple(int(c) for c in dominant)

In [15]:
def classify_clothing_type(image_path):
    """Classifies clothing as solid-colored, gradient, or patterned."""
    clothing_pixels = ignore_alpha(image_path)  # Load and extract clothing pixels

    unique ,num_dominant_colors = dominant_colors_kmeans(clothing_pixels, k=10)
    
    print(unique)
    return num_dominant_colors

In [16]:
def byColorThief(image):
    ct = ColorThief(image)
    dominant = ct.get_color(quality=1)

    palette =ct.get_palette(color_count=5)

    return dominant

# Main

In [19]:
image = "Inputs/blacknwhite.png"
#result = classify_clothing_type(image)
result = byColorThief(image)

print(result)

color=small_find_closest_color(*result)
print(color)

#color=big_find_closest_color(*result)
#print(color)

(23, 23, 24)
Black
